# Trabajo Práctico 2 - Aprendizaje Automático 2019 - Comisión 1

## Importación de las librería necesarias

* sklearn: https://scikit-learn.org/stable/ Machine Learning in Python
* pandas: https://pandas.pydata.org/ Python Data Analysis Library
* csv: https://realpython.com/python-csv/ Reading and Writing CSV Files in Python
* pydotplus: https://pypi.org/project/pydotplus/ Python Interface to Graphviz’s Dot language
    Se debe instalar https://www.graphviz.org/ y actualizar Anaconda
* matplotlib: https://matplotlib.org/  Python 2D plotting library


In [38]:
import pandas as pd
import csv
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

## Leer archivo csv (newdata.csv)
Recorre todas las filas del archivo csvfile y lo deja en lista a, en donde cada fila es una lista 

In [39]:
data_full = []
attributes_location=set()
attributes_description=set()
attributes_url=set()
attributes_verified=set()
attributes_lan=set()
attributes_default_profile=set()
attributes_default_profile_image=set()
attributes_has_extended_profile=set()
attributes_full=[]

with open('../data/newdata.csv','r', encoding="utf8") as csv_file:
    reader = csv.reader(csv_file)
    headers = next(reader,None)
    for row in reader:
        data_full.append (row)
        attributes_location.add(row[0])
        attributes_description.add(row[1])
        attributes_url.add(row[2])
        attributes_verified.add(row[8])
        attributes_lan.add(row[10])
        attributes_default_profile.add(row[11])
        attributes_default_profile_image.add(row[12])
        attributes_has_extended_profile.add(row[13])
        
attributes_full.append(list(attributes_location))
attributes_full.append(list(attributes_description))
attributes_full.append(list(attributes_url))
attributes_full.append(list(attributes_verified))
attributes_full.append(list(attributes_lan))
attributes_full.append(list(attributes_default_profile))
attributes_full.append(list(attributes_default_profile_image))
attributes_full.append(list(attributes_has_extended_profile))


print("Cantidad de Elementos: ",len(data_full))
print("Cantidad de Columnas a traducir: ",len(attributes_full))

Cantidad de Elementos:  2770
Cantidad de Columnas a traducir:  8


## Mapeo de categoricos

En la matriz ``` data_labeled ``` quedan los datos de entrenamiento.

El array ```y_vector_class ``` representa la clase a la que pertenece cada fila del array.

In [40]:
def process_to_category(data_raw, attributes_to_category):
    num_attributes=len(attributes_full)
    y_vector_class=[]
    data_labeled=[]

    le1 = preprocessing.LabelEncoder()
    le2 = preprocessing.LabelEncoder()
    le3 = preprocessing.LabelEncoder()
    le4 = preprocessing.LabelEncoder()
    le5 = preprocessing.LabelEncoder()
    le6 = preprocessing.LabelEncoder()
    le7 = preprocessing.LabelEncoder()
    le8 = preprocessing.LabelEncoder()
    le1.fit(attributes_full[0])
    le2.fit(attributes_full[1])
    le3.fit(attributes_full[2])
    le4.fit(attributes_full[3])
    le5.fit(attributes_full[4])
    le6.fit(attributes_full[5])
    le7.fit(attributes_full[6])
    le8.fit(attributes_full[7])

    for i in range(0,len(data_raw)):
        c0=le1.transform(data_raw[i][0:1])[0]
        c1=le2.transform(data_raw[i][1:2])[0]
        c2=le3.transform(data_raw[i][2:3])[0]
        c3=data_raw[i][3:4][0]
        c4=data_raw[i][4:5][0]
        c5=data_raw[i][5:6][0]
        c6=data_raw[i][6:7][0]
        c7=data_raw[i][7:8][0]
        c8=le4.transform(data_raw[i][8:9])[0]
        c9=data_raw[i][9:10][0]
        c10=le5.transform(data_raw[i][10:11])[0]
        c11=le6.transform(data_raw[i][11:12])[0]
        c12=le7.transform(data_raw[i][12:13])[0]
        c13=le8.transform(data_raw[i][13:14])[0]
        # Hay que armar C no solo con los traducibles sino tambien con los numericos.
        data_labeled.append([c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13])

    # Separo la ultima columna como vector de resultado.
    for i in range(0,len(data_raw)):
        y_vector_class.append(data_raw[i][len(data_raw[1])-1])
    
    return data_labeled, y_vector_class

## Separación en conjuntos de entrenamiento y test

``` original_train_x, original_train_y ``` contienen el conjunto principal de los datos de entrenamiento.

``` test_X, test_Y ``` son los datos de test que no se van a utilizar hasta el final, para calcular el accuracy de los algoritmos.

In [41]:
data_labeled, y_vector_class = process_to_category(data_raw=data_full, attributes_to_category=attributes_full)

# Se separan los datos en dataFrame
# Creo el Data Frame para comenzar a separar en conjuntos de datos
dataFrame=pd.DataFrame(data_labeled)

original_train_X, test_X, original_train_y, test_y = train_test_split(dataFrame, y_vector_class, 
                                                   train_size=0.7,
                                                   test_size=0.3,
                                                   random_state=123)

print("train_X: " , len(original_train_X))
print("train_y: " ,len(original_train_y))
print("test_y: " ,len(test_y))
print("test_X: " ,len(test_X))

train_X, validate_X, train_y, validate_y = train_test_split(original_train_X, original_train_y, 
                                                   train_size=0.8,
                                                   test_size=0.2,
                                                   random_state=123)

print("------------------------")
print("train_X: " , len(train_X))
print("train_y: " ,len(train_y))
print("validate_X: " ,len(validate_X))
print("validate_y: " ,len(validate_y))

train_X:  1938
train_y:  1938
test_y:  831
test_X:  831
------------------------
train_X:  1550
train_y:  1550
validate_X:  388
validate_y:  388


#### Calculo de scores

In [42]:
def calculate_scores(validations, predictions):
    acc = accuracy_score(validations, predictions)
    tn, fp, fn, tp = confusion_matrix(validations, predictions).ravel()
    prec = tp / (tp + fp)
    recall = tp / ( tp + fn )
    f1 = 2 * ((prec * recall)/(prec + recall))
    return acc, tp, fp, fn, tp, prec, recall, f1

#### Set del cross validations
Si cross_validation == 2, entonces simplemente hace un train y validation

Si cross_validation == 5 realiza el clasico k-cross-validation con k == 5

In [43]:
cross_validation = 5

### Random Forest

In [44]:
estimators = [10, 50, 100]
max_depth = [2, 3, 4, 5, None]
quality = ["gini", "entropy"]
max_features = ["sqrt", "log2", None]
bootstrap = [True, False]

params_rf = {'n_estimators': estimators, 
             'max_depth': max_depth, 
             'criterion': quality,
             'max_features': max_features,
             'bootstrap': bootstrap}

random_forest_grid_search = GridSearchCV(RandomForestClassifier(), params_rf, cv=cross_validation)
random_forest_model = random_forest_grid_search.fit(original_train_X, original_train_y)

random_forest_best = random_forest_grid_search.best_estimator_
random_forest_pred = random_forest_model.predict(test_X)
rf_acc, rf_tn, rf_fp, rf_fn, rf_tp, rf_prec, rf_recall, rf_f1 = calculate_scores(test_y, random_forest_pred)
print(random_forest_best)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=None, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


### Decision Tree

In [45]:
params_dt = {'max_depth': [3, 6, None], 'max_leaf_nodes':[None], 'criterion':["gini", "entropy"]}

decision_tree_grid_search = GridSearchCV(DecisionTreeClassifier(), params_dt, cv=cross_validation)
decision_tree_model = decision_tree_grid_search.fit(original_train_X, original_train_y)

decision_tree_best = decision_tree_model.best_estimator_
decision_tree_pred = decision_tree_model.predict(test_X)
dt_acc, dt_tn, dt_fp, dt_fn, dt_tp, dt_prec, dt_recall, dt_f1 = calculate_scores(test_y, decision_tree_pred)
print (decision_tree_best)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=6,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')


### KNN

In [46]:
params_knn = {'n_neighbors': np.arange(1, 25), 'weights': ['uniform', 'distance'], 
              'algorithm': ['ball_tree', 'kd_tree', 'brute', 'brute']}

knn_grid_search = GridSearchCV(KNeighborsClassifier(), params_knn, cv=cross_validation)
knn_model = knn_grid_search.fit(original_train_X, original_train_y)

knn_best = knn_model.best_estimator_
knn_pred = knn_grid_search.predict(test_X)
knn_acc, knn_tn, knn_fp, knn_fn, knn_tp, knn_prec, knn_recall, knn_f1 = calculate_scores(test_y, knn_pred)
print(knn_best)

KNeighborsClassifier(algorithm='ball_tree', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=7, p=2,
                     weights='uniform')


### Regresion logistica

In [47]:
log_reg = LogisticRegression()
logistic_model = log_reg.fit(original_train_X, original_train_y)

lr_pred = logistic_model.predict(test_X)
lr_acc, lr_tn, lr_fp, lr_fn, lr_tp, lr_prec, lr_recall, lr_f1 = calculate_scores(test_y, lr_pred)

/home/zero/.conda/aprendizaje2/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


### Votacion

In [48]:
estimators=[('knn', knn_best), ('dt', decision_tree_best), ('log_reg', log_reg)]
ensemble = VotingClassifier(estimators, voting='hard')

ens_model = ensemble.fit(train_X, train_y)
ensemble_score = ensemble.score(test_X, test_y)

ens_y_pred = ens_model.predict(test_X)

ens_acc, ens_tn, ens_fp, ens_fn, ens_tp, ens_prec, ens_recall, ens_f1 = calculate_scores(test_y, ens_y_pred)

/home/zero/.conda/aprendizaje2/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


## ADA BOOST

In [49]:
abc = AdaBoostClassifier(n_estimators=50,
                         learning_rate=1,base_estimator=decision_tree_best)

ada_model = abc.fit(original_train_X, original_train_y)

ada_y_pred = ada_model.predict(test_X)
ada_acc, ada_tn, ada_fp, ada_fn, ada_tp, ada_prec, ada_recall, ada_f1 = calculate_scores(test_y,ada_y_pred)

### Gravo las medidas

In [50]:
csv.register_dialect('my_dialect', delimiter = ';', quotechar='\"')
with open("./scores.csv", "w") as csv_file:
    writer = csv.writer(csv_file, dialect='my_dialect')
    writer.writerow(["method","accuracy","tn","fp","fn","tp","prec","recall","f1"])
    writer.writerow(["RandomForest",rf_acc, rf_tn, rf_fp, rf_fn, rf_tp, rf_prec, rf_recall, rf_f1])
    writer.writerow(["DecisionTree",dt_acc, dt_tn, dt_fp, dt_fn, dt_tp, dt_prec, dt_recall, dt_f1])
    writer.writerow(["KNN",knn_acc, knn_tn, knn_fp, knn_fn, knn_tp, knn_prec, knn_recall, knn_f1])
    writer.writerow(["LogisticRegression",lr_acc, lr_tn, lr_fp, lr_fn, lr_tp, lr_prec, lr_recall, lr_f1])
    writer.writerow(["Ensamble",ens_acc, ens_tn, ens_fp, ens_fn, ens_tp, ens_prec, ens_recall, ens_f1])
    writer.writerow(["AdaBoost",ada_acc, ada_tn, ada_fp, ada_fn, ada_tp, ada_prec, ada_recall, ada_f1])

#### Scores

In [51]:
print('knn: {}'.format(knn_best.score(test_X,test_y)))
print('dt: {}'.format(random_forest_best.score(test_X, test_y)))
print('log_reg: {}'.format(log_reg.score(test_X, test_y)))
print('rf: {}'.format(random_forest_best.score(test_X, test_y)))
print('ada: {}'.format(abc.score(test_X, test_y)))

knn: 0.7087845968712395
dt: 0.8892900120336944
log_reg: 0.7316486161251504
rf: 0.8892900120336944
ada: 0.8832731648616126
